<a href="https://colab.research.google.com/github/verdwis/Text-Data-Augmentation-With-EDA/blob/main/EDA__embedding_Indobert_and_postagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Instal library

In [1]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.5 MB/s eta 0:00:00


In [3]:
!pip install python-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 15.4 MB/s eta 0:00:00


# 2. Import Library

In [43]:
import torch
import random
import string
import nltk
import pandas as pd

from transformers import AutoTokenizer, AutoModelForMaskedLM
from nltk.tag import CRFTagger
from nltk.corpus import stopwords
from sklearn import preprocessing

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
model_name = "indolem/indobert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# 3. Synonym Replacement

In [26]:
def synonym_replacement(text):
    # Split kalimat menjadi array kata
    words = text.split()
    # print('kata asli = ', words)

    # Pilih kata secara acak dengan pos tag yang sesuai
    def choose_random_word(words):
        while True:
            word = random.choice(words)
            postag = postag_word(word)
            if postag in ['VB', 'JJ', 'RB', 'NN']:
                return word

    # Proses postagging menggunakan CRFTagger
    def postag_word(word):
        tagger = CRFTagger()
        tagger.set_model_file('/content/drive/MyDrive/Tesis_Very_2022/EDA/EDA/all_indo_man_tag_corpus_model.crf.tagger')
        tagged = tagger.tag_sents([[(word)]])
        return tagged[0][0][1]

    selected_word = choose_random_word(words)
    # print('kata yang di masking = ', selected_word)

    # Ganti kata terpilih dengan "[MASK]"
    masked_text = text.replace(selected_word, "[MASK]")
    # print('ganti kata yang di pilih dengan [MASK] = ', masked_text)

    # Prediksi kata pengganti "[MASK]" berdasarkan konteks dalam kalimat

    tokenized_text = tokenizer.tokenize(masked_text)
    # print('token teks =', tokenized_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # print('index token =', indexed_tokens)
    token_tensor = torch.tensor([indexed_tokens])
    # print('token tensor = ', token_tensor)

    with torch.no_grad():
        outputs = model(token_tensor)
        # print('output pred tensor = ', outputs)
        predictions = outputs.logits
        # predictions = outputs[0]
    # print('hasil prediksi', predictions)

    masked_index = tokenized_text.index("[MASK]")
    # print('masked index = ', masked_index)
    predicted_token_logits = predictions[0, masked_index]
    # print('predicted_token_logits', predicted_token_logits)
    predicted_token_ids = predicted_token_logits.topk(k=10).indices.tolist()
    # print('predicted_token_ids', predicted_token_ids)

    # Memilih kata pengganti yang merupakan kata dan bukan simbol, angka, tanda baca, emotikon, karakter, punctuation, atau stopword
    valid_predicted_token_ids = []
    stop_words = set(stopwords.words('indonesian'))

    for token_id in predicted_token_ids:
        token = tokenizer.convert_ids_to_tokens([token_id])[0]
        if token.isalpha() and token not in stop_words and not all(char in string.punctuation for char in token):
            valid_predicted_token_ids.append(token_id)

   # Jika tidak ada token yang valid, gunakan token MASK sebagai pengganti
    if len(valid_predicted_token_ids) == 0:
        valid_predicted_token_ids.append(tokenizer.mask_token_id)
    
    # Memilih kata pengganti secara acak
    replaced_token_id = random.choice(valid_predicted_token_ids)
    # print('replaced_token_id = ', replaced_token_id)
    replaced_token = tokenizer.convert_ids_to_tokens([replaced_token_id])[0]
    # print('replaced_token = ', replaced_token)
    
    # Ganti kata "[MASK]" dalam kalimat dengan kata pengganti
    replaced_text = masked_text.replace("[MASK]", replaced_token)
    # print('replaced_text = ', replaced_text)
    
    return replaced_text

In [27]:
# Contoh penggunaan
text = "aplikasi bagus guna proses antar cepat aman"
replaced_text = synonym_replacement(text)
print("Teks asli: ", text)
print("Teks hasil SR: ", replaced_text)

Teks asli:  aplikasi bagus guna proses antar cepat aman
Teks hasil SR:  aplikasi bagus guna proses antar cepat sunda


# 4. Random Insertion

In [15]:
def random_insertion(text):
    orig_text_list = text.split()
    len_input = len(orig_text_list)

    rand_idx = random.randint(1, len_input-2)
    new_text_list = orig_text_list[:rand_idx] + ['[MASK]'] + orig_text_list[rand_idx:]
    new_mask_sent = ' '.join(new_text_list)

    # Prediksi kata pengganti "[MASK]" berdasarkan konteks dalam kalimat
    tokenized_text = tokenizer.tokenize(new_mask_sent)

    # Hapus simbol, tanda baca, karakter, punctuation, emoticon, dan stop word dalam bahasa Indonesia dari token
    tokenized_text1 = [token for token in tokenized_text if token.isalpha() and token not in stopwords.words('indonesian') and not all(char in string.punctuation for char in token)]
    # print('token teks setelah filter =', tokenized_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text1)
    token_tensor = torch.tensor([indexed_tokens])

    with torch.no_grad():
        outputs = model(token_tensor)
        predictions = outputs.logits

    replaced_index = rand_idx
    predicted_token_logits = predictions[0, replaced_index]
    predicted_token_ids = predicted_token_logits.topk(k=10).indices.tolist()
    replaced_token_id = random.choice(predicted_token_ids)
    replaced_token = tokenizer.convert_ids_to_tokens([replaced_token_id])[0]

    replaced_text = new_mask_sent.replace("[MASK]", replaced_token)

    return replaced_text

In [17]:
# Contoh penggunaan
text = "aplikasi bagus berguna proses pengantaran cepat aman"
replaced_text = random_insertion(text)
print("Teks asli: ", text)
print("Teks hasil RI: ", replaced_text)

Teks asli:  aplikasi bagus berguna proses pengantaran cepat aman
Teks hasil RI:  aplikasi kasar bagus berguna proses pengantaran cepat aman


# 5. Random Swap

In [18]:
def random_swap(text):
    # Step 1: Split kalimat menjadi array kata
    words = text.split()
    # print('words = ', words)

    # Step 2: Ambil dua kata secara acak
    swap_index1 = random.randint(0, len(words) - 1)
    # print('swap_index1 = ', swap_index1)
    swap_index2 = random.randint(0, len(words) - 1)
    while swap_index2 == swap_index1:
        swap_index2 = random.randint(0, len(words) - 1)
    # print('swap_index2 = ', swap_index2)

    # Step 3: Tukar posisi dua kata tersebut
    words[swap_index1], words[swap_index2] = words[swap_index2], words[swap_index1]

    # Step 4: Hasil augmentasi
    augmented_text = " ".join(words)
    # print('hasil augmentasi = ',augmented_text)

    return augmented_text

In [19]:
# Contoh penggunaan
text = "aplikasi bagus berguna proses pengantaran cepat aman"
replaced_text = random_swap(text)
print("Teks asli: ", text)
print("Teks hasil RI: ", replaced_text)

Teks asli:  aplikasi bagus berguna proses pengantaran cepat aman
Teks hasil RI:  aplikasi bagus proses berguna pengantaran cepat aman


# 6. Random Deletion

In [28]:
def random_deletion(text):
    # Step 1: Split kalimat menjadi array kata
    words = text.split()
    # print('words = ', words)

    # Step 2: Pilih kata secara acak
    selected_word = random.choice(words)
    # print('selected_word = ', selected_word)

    # Step 3: Proses postagging dengan CRFTagger
    tagger = CRFTagger()
    tagger_path = '/content/drive/MyDrive/Tesis_Very_2022/EDA/EDA/all_indo_man_tag_corpus_model.crf.tagger'  # Path to the CRF tagger model file
    tagger.set_model_file(tagger_path)
    postags = tagger.tag_sents([words])[0]
    # print('postags = ', postags)

    while selected_word in [word for word, tag in postags if tag in ['VB', 'JJ', 'RB', 'NN']]:
        selected_word = random.choice(words)
        # print('selected_word = ', selected_word)

    # Step 4: Hapus kata yang dipilih
    words.remove(selected_word)

    # Step 5: Hasil augmentasi
    augmented_text = " ".join(words)

    return augmented_text

In [29]:
text = "ini adalah contoh kalimat untuk pengujian random deletion."
augmented_text = random_deletion(text)
print("Kalimat asli:", text)
print("Hasil augmentasi:", augmented_text)

Kalimat asli: ini adalah contoh kalimat untuk pengujian random deletion.
Hasil augmentasi: ini adalah contoh kalimat pengujian random deletion.


# 7. Model EDA

In [35]:
def create_augmentation(data, operation):
    n = len(data)
    data_copy = data.copy()
    for i in range(n):
        if operation.lower() == 'sr':
            data_copy.at[i, 'text'] = synonym_replacement(data.at[i, 'text'])
        if operation.lower() == 'ri':
            data_copy.at[i, 'text'] = random_insertion(data.at[i, 'text'])
        if operation.lower() == 'rs':
            data_copy.at[i, 'text'] = random_swap(data.at[i, 'text'])
        if operation.lower() == 'rd':
            data_copy.at[i, 'text'] = random_deletion(data.at[i, 'text'])
    return data_copy

In [38]:
def augment_data(data, k, operation):
    data_copy = data.copy()
    for i in range(k):
        df = create_augmentation(data, operation)
        data_copy = pd.concat([data_copy, df], ignore_index = True)
    return data_copy.drop_duplicates()

# 8. Persiapan Data

In [40]:
#load dataset
df_train = pd.read_csv('/content/drive/MyDrive/Tesis_Very_2022/FIKS INDO NLU/dataset/preprocessing/df_train_indonlu_clean.csv')
df_train.head()

,text,label
0,warung milik usaha pabrik puluh kenal putih ba...,positive
1,mohon ulama lurus k ambai hujah partai lah sua...,neutral
2,lokasi strategis jalan sumatera bandung nya ny...,positive
3,betapa bahagia nya unboxing paket barang nya b...,positive
4,duh mahasiswa sombong kasih kartu kuning ajar ...,negative


In [41]:
#ambil 50 record sebagai contoh
df_train = df_train[:50]

In [44]:
# Labeling Label dataset
label_encoder = preprocessing.LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train['label'])
df_train.head()

,text,label
0,warung milik usaha pabrik puluh kenal putih ba...,2
1,mohon ulama lurus k ambai hujah partai lah sua...,1
2,lokasi strategis jalan sumatera bandung nya ny...,2
3,betapa bahagia nya unboxing paket barang nya b...,2
4,duh mahasiswa sombong kasih kartu kuning ajar ...,0


In [45]:
#cek distribusi data
df_train.label.value_counts()

2    30
0    14
1     6
Name: label, dtype: int64

dataset netal

In [46]:
#definisikan label netral untuk proses augmentasi
df_neutral = df_train[df_train['label']==1]

In [48]:
#simpan dataset label netral
df_neutral.to_csv('/content/df_neutral.csv', index=False)

In [51]:
#dataset netral
df_neutral = pd.read_csv('/content/df_neutral.csv')
df_neutral.head(5)

,text,label
0,mohon ulama lurus k ambai hujah partai lah sua...,1
1,bum bentuk ucu bentuk ucu,1
2,pil riwan kamil menang baju merah,1
3,datang presiden ri rombong gaet t terminal t s...,1
4,risa jatuh pilih nya gus idul puti pilau yatim,1


dataset positif

In [59]:
#definisikan label netral untuk proses augmentasi
df_positif = df_train[df_train['label']==2]

In [60]:
#simpan dataset label positif
df_positif.to_csv('/content/df_positif.csv', index=False)

dataset negarif

In [61]:
#definisikan label netral untuk proses augmentasi
df_negatif = df_train[df_train['label']==0]

In [62]:
#simpan dataset label negatif
df_negatif.to_csv('/content/df_negatif.csv', index=False)

# 9. Proses EDA

In [52]:
# EDA SR
eda_sr = augment_data(df_neutral, 2, 'sr')

In [54]:
eda_sr.label.value_counts()

1    16
Name: label, dtype: int64

In [55]:
eda_sr.to_csv('/content/eda_sr.csv', index=False)

# 10. Concat dataset

In [63]:
df_positif = pd.read_csv('/content/df_positif.csv')

In [64]:
df_negatif = pd.read_csv('/content/df_negatif.csv')

In [65]:
df_netral = pd.read_csv('/content/eda_sr.csv')
df_netral.shape

(16, 2)

In [66]:
eda_sr = pd.concat([df_positif, df_negatif, df_netral], ignore_index=True, sort=False)

In [67]:
eda_sr.label.value_counts()

2    30
1    16
0    14
Name: label, dtype: int64

# 11. Save Dataset

In [69]:
eda_sr.to_csv('/content/eda_sr.csv', index=False)